# Download this file on your local computer


In [1]:
import pandas as pd
import numpy as np
    
import xml.etree.ElementTree as ET

from datetime import datetime 
pd.set_option('display.max_columns', None)

In [2]:
def extract_station_info(data, node, prefix):
  if node:
    data[prefix + 'Code'].append(node.find('./ns2:StationCode', ns).text)
    data[prefix + 'UIC'].append(int(node.find('./ns2:UICCode', ns).text))
    data[prefix + 'Type'].append(int(node.find('./ns2:Type', ns).text))
  else:
    data[prefix + 'Code'].append(np.NaN)
    data[prefix + 'UIC'].append(np.NaN)
    data[prefix + 'Type'].append(np.NaN)

def parse_timestamp(date_string):
  return datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S.%fZ')

def encode_list(nodes, sufix):
  UIC_codes = [node.find(sufix, ns).text for node in nodes]
  return ';'.join(UIC_codes)

In [3]:
# namespace of the xml object
ns={
    'ns1d': 'urn:ndov:cdm:trein:reisinformatie:messages:5',
    'ns1a': 'urn:ndov:cdm:trein:reisinformatie:messages:dynamischeaankomststaat:1',
    'ns2': 'urn:ndov:cdm:trein:reisinformatie:data:4',
}

# dictionary that will save the data while loading it in
def arrival_data_template():

    return {
        'ObservationTime': [],

        # Ride
        'RideId':[],
        'RideTime': [],

        # Departure station
        'DepartureStationCode': [],
        'DepartureStationUIC': [],
        'DepartureStationType': [],

        # Train
        'TrainId': [],
        'TrainType': [],
        'TrainOperator': [],

        # Actual destination
        'DestinationStationCode': [],
        'DestinationStationUIC': [],
        'DestinationStationType': [],

        # Arrival times
        'PlannedArrivalTime': [],
        'ActualArrivalTime': [],

        # Arrival platform
        'PlannedArrivalPlatform': [],
        'PlannedArrivalPlatformSuffix': [],
        'ActualArrivalPlatform': [],
        'ActualArrivalPlatformSuffix': [],

#         # Departure platforms
#         'PlannedDeparturePlatform': [],
#         'ActualDeparturePlatform': [],

#         # Stop stations
#         'PlannedStopStations': [],
#         'ActualStopStations': [],

#         # Matirial type
#         'MaterialType': [],
#         'MaterialDesignation': [],
#         'MaterialLength': [],

#         'ChangeType': [],

    }

def departure_data_template():
    return {
    'ObservationTime': [],

    # Ride
    'RideId':[],
    'RideTime': [],

    # Departure station
    'DepartureStationCode': [],
    'DepartureStationUIC': [],
    'DepartureStationType': [],

    # Train
    'TrainId': [],
    'TrainType': [],
    'TrainOperator': [],

    # Actual destination
    'DestinationStationCode': [],
    'DestinationStationUIC': [],
    'DestinationStationType': [],

    # Departure times
    'PlannedDepartureTime': [],
    'ActualDepartureTime': [],

    # Departure platform
    'PlannedDeparturePlatform': [],
    'PlannedDeparturePlatformSuffix': [],
    'ActualDeparturePlatform': [],
    'ActualDeparturePlatformSuffix': [],

    # Departure platforms
    'PlannedDeparturePlatform': [],
    'ActualDeparturePlatform': [],

    # Stop stations
    'PlannedStopStations': [],
    'ActualStopStations': [],

    # Matirial type
    'MaterialType': [],
    'MaterialDesignation': [],
    'MaterialLength': [],
    
    'HasChange': [],
    'ChangeType': [],

}
  


In [4]:
station_amsterdam = b'<ns2:RitStation><ns2:StationCode>ASD</ns2:StationCode>'
station_utrecht = b'<ns2:RitStation><ns2:StationCode>UT</ns2:StationCode>'
source_amsterdam = b'<ns2:TreinHerkomst InfoStatus=""Gepland""><ns2:StationCode>ASD</ns2:StationCode'
source_utrecht = b'<ns2:TreinHerkomst InfoStatus=""Gepland""><ns2:StationCode>UT</ns2:StationCode>'
destination_amsterdam = b'<ns2:TreinEindBestemming InfoStatus=""Gepland""><ns2:StationCode>AMS</ns2:StationCode>'
destination_utrecht = b'<ns2:TreinEindBestemming InfoStatus=""Gepland""><ns2:StationCode>UT</ns2:StationCode>'

In [5]:
import requests
import lzma

arrival_lines = []
departure_lines = []

parsing_days = pd.date_range(start='2019/12/1', end='2019/12/31').strftime('%Y-%m-%d').values
print(parsing_days)

for day in parsing_days:
    print('downloading: ',day) 
    url = f'https://trein.fwrite.org/AMS-Aurora-archive/{day[:7]}/DVSPPV_{day}.csv.xz'

    req = requests.get(url, stream=True)

    with lzma.LZMAFile(req.raw) as file:
        for line in file:
            if (source_utrecht in line and station_amsterdam in line) or (source_amsterdam in line and station_utrecht in line) :
                arrival_lines.append(line)
            if (station_amsterdam in line and destination_utrecht in line) or (station_utrecht in line and destination_amsterdam in line) :
                departure_lines.append(line)
    

['2019-12-01' '2019-12-02' '2019-12-03' '2019-12-04' '2019-12-05'
 '2019-12-06' '2019-12-07' '2019-12-08' '2019-12-09' '2019-12-10'
 '2019-12-11' '2019-12-12' '2019-12-13' '2019-12-14' '2019-12-15'
 '2019-12-16' '2019-12-17' '2019-12-18' '2019-12-19' '2019-12-20'
 '2019-12-21' '2019-12-22' '2019-12-23' '2019-12-24' '2019-12-25'
 '2019-12-26' '2019-12-27' '2019-12-28' '2019-12-29' '2019-12-30'
 '2019-12-31']
downloading:  2019-12-01


KeyboardInterrupt: 

In [6]:
from io import StringIO

df_arrivals = pd.read_csv(StringIO(b'\n'.join(arrival_lines).decode('utf-8')), header=None, names=['date', 'xml_obj', 'uuid'])
df_departures = pd.read_csv(StringIO(b'\n'.join(departure_lines).decode('utf-8')), header=None, names=['date', 'xml_obj', 'uuid'])

In [7]:
df_departures.to_csv('2019-12-01--2019-12-31-departures.csv', index=None)
df_arrivals.to_csv('2019-12-01--2019-12-31-arrivals.csv', index=None)

# Please keep in mind this is memory intensive good approach is to do each month seperately and later merge them

In [8]:
dfa_temp = pd.read_csv('../assets/data/2019 UT-ASD/2019-01-01--2019-03-31 arrivals.csv')  

In [9]:
arrival_data = arrival_data_template()

# loop through every day

for root_text, observation_time in dfa_temp[['xml_obj', 'date']].values:  
            
    arrival_data['ObservationTime'].append(observation_time)

    root = ET.fromstring(root_text)

    # Ride
    ride = root.find('./ns2:ReisInformatieProductDAS/ns2:DynamischeAankomstStaat', ns)

    arrival_data['RideId'].append(int(ride.find('./ns2:RitId', ns).text))
    arrival_data['RideTime'].append(root.find('./ns2:ReisInformatieProductDAS/ns2:RIPAdministratie/ns2:ReisInformatieTijdstip', ns).text)

    # Stations
    train = ride.find('./ns2:TreinAankomst', ns)

    destinationStation = ride.find('./ns2:RitStation', ns)
    departureStation = train.find('./ns2:TreinHerkomst[@InfoStatus="Gepland"]',ns)
    
    extract_station_info(arrival_data, departureStation, 'DepartureStation')
    extract_station_info(arrival_data, destinationStation, 'DestinationStation')

    # Arrival times
    arrival_data['ActualArrivalTime'].append(train.find('./ns2:AankomstTijd[@InfoStatus="Actueel"]', ns).text)
    arrival_data['PlannedArrivalTime'].append(train.find('./ns2:AankomstTijd[@InfoStatus="Gepland"]', ns).text)
    
    # Train
    arrival_data['TrainId'].append(train.find('./ns2:TreinNummer', ns).text)
    arrival_data['TrainType'].append(train.find('./ns2:TreinSoort', ns).text)
    arrival_data['TrainOperator'].append(train.find('./ns2:Vervoerder', ns).text)

    arrival_data['PlannedArrivalPlatform'].append(train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Gepland"]/ns2:SpoorNummer', ns).text)
    suffix = train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Gepland"]/ns2:SpoorFase', ns)
    arrival_data['PlannedArrivalPlatformSuffix'].append(None if suffix is None else suffix.text)

    arrival_data['ActualArrivalPlatform'].append(train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Actueel"]/ns2:SpoorNummer', ns).text)
    suffix = train.find('./ns2:TreinAankomstSpoor[@InfoStatus="Actueel"]/ns2:SpoorFase', ns)
    arrival_data['ActualArrivalPlatformSuffix'].append(None if suffix is None else suffix.text)
    
# Convert the dictionary to a dataframe
dfa = pd.DataFrame(arrival_data)    
print(dfa.shape) # show the file size kinda
# # save the file, specify your path and name for the file
# df_flat.to_csv('./data/september.csv.zip', 
#                index=False, 
#                compression=dict(method='zip', archive_name='january.csv'))



(7146, 18)


In [10]:
dfd_temp = pd.read_csv('../assets/data/2019 UT-ASD/2019-01-01--2019-03-31 departures.csv')

In [11]:
departure_data = departure_data_template()
# Parse records about departures
for root_text, observation_time in dfd_temp[['xml_obj', 'date']].values:  

    departure_data['ObservationTime'].append(observation_time)

    root = ET.fromstring(root_text)

    # Ride
    ride = root.find('./ns2:ReisInformatieProductDVS/ns2:DynamischeVertrekStaat', ns)

    departure_data['RideId'].append(int(ride.find('./ns2:RitId', ns).text))
    departure_data['RideTime'].append(root.find('./ns2:ReisInformatieProductDVS/ns2:RIPAdministratie/ns2:ReisInformatieTijdstip', ns).text)

    # Stations
    train = ride.find('./ns2:Trein', ns)

    departureStation = ride.find('./ns2:RitStation', ns)
    destinationStation = train.find('./ns2:TreinEindBestemming[@InfoStatus="Gepland"]',ns)
    
    extract_station_info(departure_data, departureStation, 'DepartureStation')
    extract_station_info(departure_data, destinationStation, 'DestinationStation')

    # Departure times
    departure_data['ActualDepartureTime'].append(train.find('./ns2:VertrekTijd[@InfoStatus="Actueel"]', ns).text)
    departure_data['PlannedDepartureTime'].append(train.find('./ns2:VertrekTijd[@InfoStatus="Gepland"]', ns).text)
    
    # Train
    departure_data['TrainId'].append(train.find('./ns2:TreinNummer', ns).text)
    departure_data['TrainType'].append(train.find('./ns2:TreinSoort', ns).text)
    departure_data['TrainOperator'].append(train.find('./ns2:Vervoerder', ns).text)
    
    platform_planned = train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Gepland"]', ns)
    platform_actual = train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Actueel"]', ns)
    
    if platform_planned and platform_actual:   
        departure_data['PlannedDeparturePlatform'].append(train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Gepland"]/ns2:SpoorNummer', ns).text)
        suffix = train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Gepland"]/ns2:SpoorFase', ns)
        departure_data['PlannedDeparturePlatformSuffix'].append(None if suffix is None else suffix.text)

        departure_data['ActualDeparturePlatform'].append(train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Actueel"]/ns2:SpoorNummer', ns).text)
        suffix = train.find('./ns2:TreinVertrekSpoor[@InfoStatus="Actueel"]/ns2:SpoorFase', ns)
        departure_data['ActualDeparturePlatformSuffix'].append(None if suffix is None else suffix.text)
    else:
        departure_data['PlannedDeparturePlatform'].append(None)
        suffix = None
        departure_data['PlannedDeparturePlatformSuffix'].append(None if suffix is None else suffix.text)

        departure_data['ActualDeparturePlatform'].append(None)
        suffix = None
        departure_data['ActualDeparturePlatformSuffix'].append(None if suffix is None else suffix.text)
    
    # Stop stations
    wagons = train.find('./ns2:TreinVleugel', ns)
    if not wagons: print('No wagons, wtf?')
    stop_stations = wagons.findall('./ns2:StopStations[@InfoStatus="Gepland"]/ns2:Station', ns)
    if not stop_stations: print('No stop_stations, wtf?')

    departure_data['PlannedStopStations'].append(encode_list(stop_stations, './ns2:UICCode'))

    stop_stations = wagons.findall('./ns2:StopStations[@InfoStatus="Actueel"]/ns2:Station', ns)
    departure_data['ActualStopStations'].append(encode_list(stop_stations, './ns2:UICCode'))
    if not stop_stations: print('No stop_stations2, wtf?')

    # Material
    material = wagons.find('./ns2:MaterieelDeelDVS', ns)
    if material:
        departure_data['MaterialType'].append(material.find('./ns2:MaterieelSoort', ns).text)
        departure_data['MaterialDesignation'].append(material.find('./ns2:MaterieelAanduiding', ns).text)
        departure_data['MaterialLength'].append(material.find('./ns2:MaterieelLengte', ns).text)
    else:
        departure_data['MaterialType'].append(np.NaN)
        departure_data['MaterialDesignation'].append(np.NaN)
        departure_data['MaterialLength'].append(np.NaN)

    # Change
    changes = root.findall('./ns2:Wijziging', ns)
    if changes:
        departure_data['HasChange'].append(True)
        departure_data['ChangeType'].append(encode_list(changes, './ns2:WijzigingType'))
    else:
        departure_data['HasChange'].append(False)
        departure_data['ChangeType'].append(np.NaN)
        
dfd = pd.DataFrame(departure_data)  
print(dfa.shape)

(7146, 18)


In [ ]:
dfa.value_counts('RideId', 'Planned')

In [25]:
if 'RideInstance' in dfd:
    del dfd['RideInstance']
dfd.insert(0, 'RideInstance', dfd.RideId.astype(str) + '#' + dfd.RideTime )

if 'RideInstance' in dfa:
    del dfa['RideInstance']
dfa.insert(0, 'RideInstance', dfa.RideId.astype(str) + '#' + dfa.RideTime )



,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedDepartureTime,ActualDepartureTime,PlannedDeparturePlatform,PlannedDeparturePlatformSuffix,ActualDeparturePlatform,ActualDeparturePlatformSuffix,PlannedStopStations,ActualStopStations,MaterialType,MaterialDesignation,MaterialLength,HasChange,ChangeType
2,1402#2019-01-01T00:18:00.000Z,2019-01-01 01:18:41.841361+01:00,1402,2019-01-01T00:18:00.000Z,ASD,8400058,6,1402,Intercity,NS,UT,8400621,6,2019-01-01T00:18:00.000Z,2019-01-01T00:18:00.000Z,2,b,2,None,8400074;8400621,8400074;8400621,VIRM,6,16210,False,NaN
8,1406#2019-01-01T01:19:00.000Z,2019-01-01 02:21:06.626009+01:00,1406,2019-01-01T01:19:00.000Z,ASD,8400058,6,1406,Intercity,NS,UT,8400621,6,2019-01-01T01:19:00.000Z,2019-01-01T01:20:16.000Z,2,b,2,b,8400621,8400621,VIRM,4,10860,False,NaN
11,1410#2019-01-01T02:19:00.000Z,2019-01-01 03:20:25.483579+01:00,1410,2019-01-01T02:19:00.000Z,ASD,8400058,6,1410,Intercity,NS,UT,8400621,6,2019-01-01T02:19:00.000Z,2019-01-01T02:19:00.000Z,2,b,2,b,8400621,8400621,VIRM,6,16210,False,NaN
14,1414#2019-01-01T03:19:00.000Z,2019-01-01 04:19:50.604934+01:00,1414,2019-01-01T03:19:00.000Z,ASD,8400058,6,1414,Intercity,NS,UT,8400621,6,2019-01-01T03:19:00.000Z,2019-01-01T03:19:00.000Z,2,b,2,b,8400621,8400621,VIRM,4,10860,False,NaN
17,1418#2019-01-01T04:19:00.000Z,2019-01-01 05:19:47.184006+01:00,1418,2019-01-01T04:19:00.000Z,ASD,8400058,6,1418,Intercity,NS,UT,8400621,6,2019-01-01T04:19:00.000Z,2019-01-01T04:19:00.000Z,2,b,2,b,8400621,8400621,VIRM,4,10860,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3348,1418#2019-03-31T03:19:00.000Z,2019-03-31 05:19:17.560193+02:00,1418,2019-03-31T03:19:00.000Z,ASD,8400058,6,1418,Intercity,NS,UT,8400621,6,2019-03-31T03:19:00.000Z,2019-03-31T03:19:00.000Z,7,a,7,a,8400621,8400621,VIRM,4,10860,False,NaN
3350,1422#2019-03-31T04:25:00.000Z,2019-03-31 06:25:39.590373+02:00,1422,2019-03-31T04:25:00.000Z,ASD,8400058,6,1422,Intercity,NS,UT,8400621,6,2019-03-31T04:25:00.000Z,2019-03-31T04:25:00.000Z,7,b,7,b,8400621,8400621,VIRM,6,16210,False,NaN
3352,7393#2019-03-31T22:28:00.000Z,2019-03-31 23:18:05.176728+02:00,7393,2019-03-31T22:28:00.000Z,ASD,8400058,6,7393,Sprinter,NS,UT,8400621,6,2019-03-31T22:28:00.000Z,2019-03-31T22:28:00.000Z,7,b,7,b,8400060;8400057;8400194;8400074;8400231;840004...,8400060;8400057;8400194;8400074;8400231;840004...,SLT,6,10050,False,NaN
3354,7395#2019-03-31T22:57:00.000Z,2019-03-31 23:47:04.005991+02:00,7395,2019-03-31T22:57:00.000Z,ASD,8400058,6,7395,Sprinter,NS,UT,8400621,6,2019-03-31T22:57:00.000Z,2019-03-31T22:57:00.000Z,2,b,2,b,8400060;8400057;8400194;8400074;8400231;840004...,8400060;8400057;8400194;8400074;8400231;840004...,SLT,6,10050,False,NaN


In [40]:
dfa = dfa.drop_duplicates('RideInstance', keep='last')
dfd = dfd.drop_duplicates('RideInstance', keep='last')
dfa.index = dfa.RideInstance
dfd.index = dfd.RideInstance

In [45]:
dfa.join(dfd, rsuffix='_Departure').RideInstance_Departure.unique()

array([nan], dtype=object)

In [56]:
dfa.loc[dfa.RideTime != dfa.PlannedArrivalTime]

,RideInstance,ObservationTime,RideId,RideTime,DepartureStationCode,DepartureStationUIC,DepartureStationType,TrainId,TrainType,TrainOperator,DestinationStationCode,DestinationStationUIC,DestinationStationType,PlannedArrivalTime,ActualArrivalTime,PlannedArrivalPlatform,PlannedArrivalPlatformSuffix,ActualArrivalPlatform,ActualArrivalPlatformSuffix
RideInstance,,,,,,,,,,,,,,,,,,,


dtype('O')

In [ ]:
dfd[['RideId','PlannedDepartureTime', 'ActualDepartureTime']]

In [28]:
dfd.ChangeType.value_counts()

dtype('int64')

In [ ]:
df = dfd

In [ ]:
df[['PlannedArrivalTime', 'ActualArrivalTime', 'PlannedArrivalPlatform', 'ActualArrivalPlatform']] = dfa[['PlannedArrivalTime', 'ActualArrivalTime', 'PlannedArrivalPlatform', 'ActualArrivalPlatform']]

In [ ]:
# you can laod the files again in 
df_july = pd.read_csv('../data/july.csv.zip')
df_august = pd.read_csv('../data/august.csv.zip')
df_september = pd.read_csv('../data/september.csv.zip')

In [ ]:
# here you merge them into one big dataframe
months = [df_july, df_august, df_september]
df_months = pd.concat(months)

In [ ]:
# save the big dataframe again
df_months.to_csv('../data/july_to_september.csv.zip', 
                 index=False, 
                 compression=dict(method='zip', archive_name='july_to_september.csv'))

In [ ]:
# here we filter out only all rides from utrecht to amsterdam and vise versa
df_ut_asd = df_months.loc[((df_months['DepartureStationCode'] == 'UT') & 
               (df_months['PlannedDestinationStationCode'] == 'ASD')) | 
              ((df_months['DepartureStationCode'] == 'ASD') & 
               (df_months['PlannedDestinationStationCode'] == 'UT'))]

In [ ]:
# save the reduced dataframe again
df_ut_asd.to_csv('../data/july_to_september_utrecht_amsterdam.csv.zip', 
                 index=False, 
                 compression=dict(method='zip', archive_name='july_to_september_utrecht_amsterdam.csv'))